In [1]:
import os

In [2]:
%pwd

'/Users/charantejakammari/Desktop/DataScience/research'

In [3]:
os.chdir("../")
%pwd

'/Users/charantejakammari/Desktop/DataScience'

# Data Ingestion

In [13]:
from dataclasses import dataclass
from pathlib import Path

@dataclass()
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
    

In [14]:
from src.Data_Science.constants import *
from src.Data_Science.utils.common import read_yaml, create_directories

In [15]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )
        return data_ingestion_config

In [ ]:
import zipfile
import urllib.request
from os.path import getsize
import os
from src.Data_Science import logger

class DataIngestion:
    def __init__(self, config):
        self.config = config

    def download_file(self):
        should_download = True

        if os.path.exists(self.config.local_data_file):
            try:
                with zipfile.ZipFile(self.config.local_data_file, 'r') as existing_zip:
                    existing_zip.testzip()
                logger.info(f"Valid file already exists of size: {getsize(self.config.local_data_file)}")
                should_download = False
            except zipfile.BadZipFile:
                logger.info("Existing file is not a valid zip. Re-downloading.")
                os.remove(self.config.local_data_file)

        if should_download:
            filename, headers = urllib.request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} downloaded!")

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        try:
            with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
                zip_ref.extractall(unzip_path)
        except zipfile.BadZipFile as exc:
            raise zipfile.BadZipFile("Downloaded file is not a valid zip. Check source_URL.") from exc


SyntaxError: invalid syntax (3770094778.py, line 34)

In [17]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-12-26 23:17:47,728: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-12-26 23:17:47,730: INFO: common: yaml file: params.yaml loaded successfully]
[2025-12-26 23:17:47,731: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-12-26 23:17:47,732: INFO: common: created directory at: artifacts]
[2025-12-26 23:17:47,733: INFO: common: created directory at: artifacts/data_ingestion]
[2025-12-26 23:17:47,901: INFO: 4018249200: artifacts/data_ingestion/data.zip downloaded!]


In [5]:
import pandas as pd

data = pd.read_csv("artifacts/data_ingestion/winequality-red.csv")

In [6]:
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In [9]:
for i in data.columns:
    print(i)

fixed acidity
volatile acidity
citric acid
residual sugar
chlorides
free sulfur dioxide
total sulfur dioxide
density
pH
sulphates
alcohol
quality


In [10]:
data.isnull().sum()

fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64

In [11]:
data.shape

(1599, 12)

# Data Validation

In [12]:
from dataclasses import dataclass
from pathlib import Path

@dataclass()
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    unzip_data_dir: Path
    all_schema: dict

In [13]:
from src.Data_Science.constants import *
from src.Data_Science.utils.common import read_yaml, create_directories

In [14]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir=config.unzip_data_dir,
            all_schema=schema,
        )

        return data_validation_config

In [11]:
import os
from src.Data_Science import logger


In [17]:
class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config

    def validate_all_columns(self) -> bool:
        try:
            validation_status = None

            data = pd.read_csv(self.config.unzip_data_dir)
            all_schema = self.config.all_schema.keys()
            all_schema_datatypes = self.config.all_schema.values()

            # loop that checks if all columns are present in the data and if all columns are of the correct datatype
            for col, datatype in zip(all_schema, all_schema_datatypes):
                if data[col].dtype != datatype:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")

            return validation_status

        except Exception as e:
            raise e

In [18]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2025-12-27 13:22:41,607: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-12-27 13:22:41,608: INFO: common: yaml file: params.yaml loaded successfully]
[2025-12-27 13:22:41,609: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-12-27 13:22:41,610: INFO: common: created directory at: artifacts]
[2025-12-27 13:22:41,610: INFO: common: created directory at: artifacts/data_validation]


# Data Transformation

In [1]:
from dataclasses import dataclass
from pathlib import Path

@dataclass()
class DataTransformationConfig:
    root_dir: Path
    data_path: Path

In [5]:
import os
os.chdir("../")

In [6]:
from src.Data_Science.constants import *
from src.Data_Science.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
        )

        return data_transformation_config

In [8]:
from sklearn.model_selection import train_test_split
import pandas as pd

In [12]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    def train_test_splitting(self):
        data = pd.read_csv(self.config.data_path)

        data = data.drop_duplicates()

        # Split the data into training and test sets. (0.75, 0.25) split.
        train, test = train_test_split(data)

        train.to_csv(os.path.join(self.config.root_dir, "train.csv"),index = False)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"),index = False)

        logger.info("Splited data into training and test sets")
        logger.info(train.shape)
        logger.info(test.shape)

        print(train.shape)
        print(test.shape)

In [13]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.train_test_splitting()
except Exception as e:
    raise e

[2025-12-27 13:49:14,982: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-12-27 13:49:14,983: INFO: common: yaml file: params.yaml loaded successfully]
[2025-12-27 13:49:14,985: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-12-27 13:49:14,985: INFO: common: created directory at: artifacts]
[2025-12-27 13:49:14,986: INFO: common: created directory at: artifacts/data_transformation]
[2025-12-27 13:49:14,998: INFO: 3107515286: Splited data into training and test sets]
[2025-12-27 13:49:14,998: INFO: 3107515286: (1019, 12)]
[2025-12-27 13:49:14,999: INFO: 3107515286: (340, 12)]
(1019, 12)
(340, 12)
